# Metropolitan Data, Assignment 4
For this exercise we have chosen a route in Amsterdam for the canal swimming event.
The distance of this swimming spot has a lenght of 5km. 
For the Municipality we found a location for the Event Headquarters which is close to the centre of the swimming route. We will quantify the number of visitors that can reach the event and the capacity for the event.

We start by importing Pandas since we need to read some files, we import other packages as well.

In [34]:
import geopy.distance
import networkx as nx
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import math
import numpy as np
import folium
import json
import os
import osmnx as ox
from shapely.geometry import Point

We will import the created GeoJSON route. Moreover, we will calculate the center point of the swimming route as a location for the Event Headquarters. 

In [35]:
# Importing made GeoJSON route (made via Geojson.io) into VSC (to make calculations)
route_geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "coordinates": [
                    [4.903764600794744, 52.37632208183015],
                    [4.9092047504975085, 52.374200241271495],
                    [4.914695271956532, 52.37616832869375],
                    [4.923308842319443, 52.37481527801236],
                    [4.927137095814004, 52.373462185876235],
                    [4.920085049903065, 52.36881830442803],
                    [4.918422781938631, 52.36712670312201],
                    [4.920739882737564, 52.36626549939223],
                    [4.921391262359208, 52.36595297476106],
                    [4.921514917273726, 52.36557544488625],
                    [4.92050506880085, 52.36477632934216],
                    [4.918913145214248, 52.36403165530362],
                    [4.91633501773822, 52.36330183484273],
                    [4.9091628735515656, 52.36566939378861],
                    [4.908729546404658, 52.36583450016303],
                    [4.90210325292847, 52.364536352345596],
                    [4.901511437210786, 52.36579629359579],
                    [4.906102007774024, 52.36673390097337],
                    [4.906917753761945, 52.36723199679929]
                ],
                "type": "LineString"
            }
        }
    ]
}

# Extract the coordinates from the GeoJSON
coordinates = route_geojson["features"][0]["geometry"]["coordinates"]

# Function to calculate the distance between two coordinates using Haversine formula (instead of using external libraries) 
def haversine(coord1, coord2):
    lon1, lat1 = coord1
    lon2, lat2 = coord2
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))

    # Radius of the Earth in kilometers (mean value)
    r = 6371

    return c * r

# Calculate the total distance of the route
total_distance = 0.0
for i in range(1, len(coordinates)):
    coord1 = coordinates[i - 1]
    coord2 = coordinates[i]
    total_distance += haversine(coord1, coord2)

# Print the total distance
print(f"Total distance of the route: {total_distance:.2f} kilometers")

# Calculate the center of the nodes
lats, lons = zip(*coordinates)
center_lat = np.mean(lats)
center_lon = np.mean(lons)

# Create a map centered around the route's center
amsterdam_map = folium.Map(location=[center_lat, center_lon], zoom_start=14)

# Retrieve waterways of Amsterdam using osmnx
ox.config(use_cache=True, log_console=True)
G = ox.graph_from_place('Amsterdam', retain_all=False, truncate_by_edge=False,
                        simplify=True, custom_filter='["waterway"~"canal"]')

# Plot the waterways on the map
ox.plot_graph_folium(G, graph_map=amsterdam_map, edge_color='green')

# Plot the route on the map
route_points = [(lat, lon) for lon, lat in coordinates]  # Swap latitude and longitude
folium.PolyLine(route_points, color="blue", weight=4.5, opacity=1).add_to(amsterdam_map)

# Mark the center coordinates
Marker_location = [center_lon,center_lat]
point= folium.Marker(Marker_location, popup='Marker Popup Text', tooltip='Marker Tooltip').add_to(amsterdam_map)
print("center_node:", Marker_location)

# Display the map
amsterdam_map


Total distance of the route: 5.04 kilometers


C:\Users\abbeh\AppData\Local\Temp\ipykernel_22172\1986816391.py:75: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True)
C:\Users\abbeh\AppData\Local\Temp\ipykernel_22172\1986816391.py:80: UserWarning: The `folium` module has been deprecated and will be removed in a future release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  ox.plot_graph_folium(G, graph_map=amsterdam_map, edge_color='green')


center_node: [52.368243121022516, 4.914239187314695]


We calculated the closest tram stops near the start and stop of the swimming route.

In [44]:
# Load tram stops GeoJSON data
tram_stops_geojson_url = 'https://maps.amsterdam.nl/open_geodata/geojson_lnglat.php?KAARTLAAG=TRAMMETRO_PUNTEN_2022&THEMA=trammetro'
tram_stops_gdf = gpd.read_file(tram_stops_geojson_url)

# Plot all tram stops on the map
for idx, stop in tram_stops_gdf.iterrows():
    folium.Marker([stop.geometry.y, stop.geometry.x], popup=stop['Naam']).add_to(amsterdam_map)

# Function to calculate the distance between two coordinates using Haversine formula
def haversine(coord1, coord2):
    lon1, lat1 = coord1
    lon2, lat2 = coord2
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))

    # Radius of the Earth in kilometers (mean value)
    r = 6371

    return c * r

# Find the closest tram stops to the start of the route
start_coords = coordinates[0]
closest_start_stop = None
min_start_distance = float('inf')

for idx, stop in tram_stops_gdf.iterrows():
    stop_coords = (stop.geometry.x, stop.geometry.y)
    distance = haversine(start_coords, stop_coords)
    if distance < min_start_distance:
        min_start_distance = distance
        closest_start_stop = stop


if not closest_start_stop.empty:  # Use .empty to check if the Series is empty
    print(f"Closest tram stop at the start: {closest_start_stop['Naam']} at a distance of {min_start_distance:.2f} km")
else:
    print("No tram stops found near the start of the route")

# Find the closest tram stops to the end of the route
end_coords = coordinates[-1]
closest_end_stop = None
min_end_distance = float('inf')

for idx, stop in tram_stops_gdf.iterrows():
    stop_coords = (stop.geometry.x, stop.geometry.y)
    distance = haversine(end_coords, stop_coords)
    if distance < min_end_distance:
        min_end_distance = distance
        closest_end_stop = stop

if not closest_end_stop.empty:
    print(f"Closest tram stop at the end: {closest_end_stop['Naam']} at a distance of {min_end_distance:.2f} km")
else:
    print("No tram stops found near the end of the route")

# Display the map
amsterdam_map


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1006)>

Finding out how many people can be transported within an hour from tramstop (Centraal Station) at start Number of trams per hour & amount of trams that stops at Centraal station

In [16]:
#Finding out how many people can be transported within an hour from tramstop (Centraal Station) at start
# Number of trams per hour & amount of trams that stops at Centraal station
trams_per_hour = 5
amount_of_trams = 6

# Capacity per tram
seating_capacity = 50
standing_capacity = 125

# Calculate the total capacity per tram
total_capacity_per_tram = seating_capacity + standing_capacity

# Calculate the total capacity per hour
total_capacity_per_hour = trams_per_hour * total_capacity_per_tram * amount_of_trams

print("Total number of people that can be transported in an hour from tramstop at start:", total_capacity_per_hour)

#Finding out how many people can be transported within an hour from tramstop (Mr. Vissersplein) at end 
# Number of trams per hour & amount of trams that stop by Mr. Vissersplein
trams_per_hour = 5
amount_of_trams = 1

# Capacity per tram
seating_capacity = 50
standing_capacity = 125

# Calculate the total capacity per tram
total_capacity_per_tram = seating_capacity + standing_capacity

# Calculate the total capacity per hour
total_capacity_per_hour = trams_per_hour * total_capacity_per_tram * amount_of_trams

print("Total number of people that can be transported in an hour from tramstop at finish:", total_capacity_per_hour)


Total number of people that can be transported in an hour from tramstop at start: 5250
Total number of people that can be transported in an hour from tramstop at finish: 875


In [17]:
from shapely.geometry import Point, LineString

# Load tram lines GeoJSON data
tram_lines_geojson_url = 'https://maps.amsterdam.nl/open_geodata/geojson_lnglat.php?KAARTLAAG=TRAMMETRO_LIJNEN_2022&THEMA=trammetro'
tram_lines_gdf = gpd.read_file(tram_lines_geojson_url)

# Define a list of tramlines to display
desired_tramlines = ['13', '14', '24', '17', '2', '26']

# Filter the tram lines GeoDataFrame to include only desired tramlines
filtered_tram_lines_gdf = tram_lines_gdf[tram_lines_gdf['Lijn'].astype(str).isin(desired_tramlines)]

# Create a map centered around Amsterdam
m = folium.Map(location=[52.377956, 4.897070], zoom_start=13)

#Plot tramlines on the map
for idx, row in tram_lines_gdf.iterrows():
    line = row['geometry']
    coords = line.xy  # Extract the coordinates of the tramline

    # Create a list of latitude and longitude pairs
    coordinates = list(zip(coords[1], coords[0]))

    # Create a PolyLine with the coordinates
    folium.PolyLine(locations=coordinates, color='purple').add_to(m)

# Display the map
m

In [21]:
import networkx as nx
import osmnx as ox
import folium

# Coordinates for my route
coordinates = route_geojson["features"][0]["geometry"]["coordinates"]

# Create a MultiDiGraph from your coordinates
G = nx.MultiDiGraph()

for i in range(len(coordinates) - 1):
    G.add_edge(i, i + 1)

# Create a GeoDataFrame for the nodes with a dummy geometry (not used)
gdf_nodes = gpd.GeoDataFrame(
    index=[start_node, finish_node, center_node],
    geometry=[None, None, None]
)

# Set the CRS to WGS 84 (epsg=4326)
gdf_nodes.crs = "EPSG:4326"

# Calculate the start, finish, and center nodes
G = ox.graph_from_place('Amsterdam, Netherlands', network_type='all')
start_node = ox.distance.nearest_nodes(G, X=coordinates[0][1], Y=coordinates[0][0])
finish_node = ox.distance.nearest_nodes(G, X=coordinates[-1][1], Y=coordinates[-1][0])
center_node = ox.distance.nearest_nodes(G, X=coordinates[len(coordinates) // 2][1], Y=coordinates[len(coordinates) // 2][0])

# Calculate the closeness centrality
start_centrality = nx.closeness_centrality(G, u=start_node)
finish_centrality = nx.closeness_centrality(G, u=finish_node)
center_centrality = nx.closeness_centrality(G, u=center_node)

# Create a folium map
m = folium.Map(location=coordinates[len(coordinates) // 2][::-1], zoom_start=15)

# Markers for the start, middle, and end nodes with centrality values
folium.Marker(list(reversed(coordinates[0])), icon=folium.Icon(color='blue'),
              popup=f'Start Centrality: {start_centrality}').add_to(m)
folium.Marker(list(reversed(coordinates[-1])), icon=folium.Icon(color='green'),
              popup=f'End Centrality: {finish_centrality}').add_to(m)
folium.Marker(list(reversed(coordinates[len(coordinates) // 2])), icon=folium.Icon(color='red'),
              popup=f'Middle Centrality: {center_centrality}').add_to(m)

# Print centrality values
print('The centrality of the start point is', start_centrality)
print('The centrality of the end point is', finish_centrality)
print('The centrality of the middle point is', center_centrality)

# Display the map
m.save("C:/Users/31621/Downloads/MSc_DATA1/centrality_map.html")
m


The centrality of the start point is 0.010759728543672943
The centrality of the end point is 0.010759728543672943
The centrality of the middle point is 0.010759728543672943


In [22]:
import pandas as pd

# Load CSV file with places data (FUNCTIEKAART from gemeente amsterdam)
places_csv_path="C:/Users/31621/Downloads/FUNCTIEKAART.csv"

# Load the places from the CSV file
places = pd.read_csv(places_csv_path)

# Define the finish point of your route
finish_point = (52.36723199679929, 4.906917753761945)


